In [2]:
import os
import sqlite3
import requests
import pandas as pd
import datetime
from dotenv import load_dotenv

In [3]:
load_dotenv()

RAPIDAPI_HOST = os.environ.get("RAPIDAPI_HOST")
RAPIDAPI_KEY = os.environ.get("RAPIDAPI_KEY")
DB_PATH = os.getenv("DB_PATH", "nfl.db")

In [ ]:
local_path = os.getcwd()
local_path

In [8]:
HEADERS = {
    "x-rapidapi-key": RAPIDAPI_KEY,
    "x-rapidapi-host": RAPIDAPI_HOST
}
BASE_URL = f"https://{RAPIDAPI_HOST}/"

In [9]:
def get(endpoint, params=None):
    r = requests.get(f"{BASE_URL}/{endpoint}", headers=HEADERS, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def run_schema(conn):
    with open("schema.sql", "r") as f:
        conn.executescript(f.read())

In [10]:
def extract_teams():
    data = get("nfl-team-listing/v1/data")
    extracted = []
    for item in data:
        team = item.get("team", {})
        extracted.append({
            "team_id": team.get("id"),
            "name": team.get("displayName"),
            "abbreviation": team.get("abbreviation"),
            "location": team.get("location")
        })
    return pd.DataFrame(extracted)

In [ ]:
df = extract_teams()
df

In [11]:
def getPlayers(team_id):
    data = get(f"nfl-player-listing/v1/data?id={team_id}")
    extracted = []
    athletes = data.get('athletes', [])
    for athlete in athletes:
        position = athlete.get('position')
        for player_info in athlete.get('items', []):
            format_string = '%Y-%m-%dT%H:%MZ'
            date_object = datetime.datetime.strptime(player_info.get("dateOfBirth"), format_string)
            extracted.append({
                "player_id": player_info.get("id"),
                "team_id": team_id,
                "firstName": player_info.get("firstName"),
                "lastName": player_info.get("lastName"),
                "unit": position,
                "position": player_info.get('position', {}).get('abbreviation'),
                "weight": player_info.get("weight"),
                "height": player_info.get("displayHeight"),
                "age": player_info.get("age"),
                "DOB": date_object.strftime('%Y-%m-%d'),
                "college": player_info.get('college', {}).get('name')
            })
    return pd.DataFrame(extracted)

def upsert_df(conn, df, table):
    if table == "players":
        pk_col = "player_id"
    elif table == "teams":
        pk_col = "team_id"
    else:
        pk_col = df.columns[0]
    df.to_sql(f"_{table}_staging", conn, if_exists="replace", index=False)
    cols = ",".join(df.columns)
    set_clause = ",".join([f"{c}=excluded.{c}" for c in df.columns])
    conn.execute(f"""
        DELETE FROM {table} 
        WHERE {pk_col} IN (SELECT {pk_col} FROM _{table}_staging)
    """)
    conn.execute(f"INSERT INTO {table} ({cols}) SELECT {cols} FROM _{table}_staging")
    conn.execute(f"DROP TABLE _{table}_staging")


In [12]:
df = getPlayers(22)
df

,player_id,team_id,firstName,lastName,unit,position,weight,height,age,DOB,college
0,5084939,22,Isaiah,Adams,offense,G,315,"6' 4""",25,2000-07-21,Illinois
1,15035,22,Kelvin,Beachum,offense,OT,308,"6' 3""",36,1989-06-08,SMU
2,2578570,22,Jacoby,Brissett,offense,QB,235,"6' 4""",32,1992-12-11,NC State
3,3128412,22,Evan,Brown,offense,C,320,"6' 3""",29,1996-09-16,SMU
4,4240657,22,Michael,Carter,offense,RB,201,"5' 8""",26,1999-05-07,North Carolina
...,...,...,...,...,...,...,...,...,...,...,...
82,3139602,22,D'Ernest,Johnson,practiceSquad,RB,208,"5' 10""",29,1996-02-27,South Florida
83,3920591,22,Nick,Leverett,practiceSquad,G,310,"6' 3""",28,1997-01-11,Rice
84,4249624,22,Nick,Muse,practiceSquad,TE,252,"6' 5""",26,1998-11-25,South Carolina
85,4875912,22,Tejhaun,Palmer,practiceSquad,WR,210,"6' 1""",25,2000-05-19,UAB


In [ ]:
print(f"Starting ETL process. DB Path: {DB_PATH}")
conn = sqlite3.connect(DB_PATH)
run_schema(conn)

# 1. Load Teams
print("Extracting and Loading Teams...")
teams_df = extract_teams()
upsert_df(conn, teams_df, "teams")
print(f"Successfully loaded {len(teams_df)} teams.")
conn.commit()
conn.close()

# 2. Load Players
all_players_df = pd.DataFrame()

print("Extracting and Loading Players...")
conn = sqlite3.connect(DB_PATH)
run_schema(conn)
for team_id in teams_df['team_id']:
    try:
        players_df = getPlayers(team_id)
        all_players_df = pd.concat([all_players_df, players_df], ignore_index=True)
        print(f" -> Loaded {len(players_df)} players for Team ID {team_id}.")
        
    except Exception as e:
        print(f"An error occurred while fetching roster for team ID {team_id}: {e}")
        continue

all_players_df.drop_duplicates(subset=['player_id'], inplace=True) 
upsert_df(conn, all_players_df, "players")
print(f"Successfully loaded {len(all_players_df)} unique players into the 'players' table.")

conn.commit()
conn.close()

In [5]:
conn = sqlite3.connect("nfl.db")

# Top fantasy performers
df = pd.read_sql("""
SELECT * FROM players
""", conn)
df

,player_id,team_id,firstName,lastName,unit,position,weight,height,age,dob,college
0,5084939,22,Isaiah,Adams,offense,Guard,315,"6' 4""",25,2000-07-21,Illinois
1,15035,22,Kelvin,Beachum,offense,Offensive Tackle,308,"6' 3""",36,1989-06-08,SMU
2,2578570,22,Jacoby,Brissett,offense,Quarterback,235,"6' 4""",32,1992-12-11,NC State
3,3128412,22,Evan,Brown,offense,Center,320,"6' 3""",29,1996-09-16,SMU
4,4240657,22,Michael,Carter,offense,Running Back,201,"5' 8""",26,1999-05-07,North Carolina
...,...,...,...,...,...,...,...,...,...,...,...
2533,4245144,28,Tay,Martin,practiceSquad,Wide Receiver,185,"6' 3""",27,1997-12-14,Oklahoma State
2534,4574814,28,Robert,McDaniel,practiceSquad,Safety,207,"6' 2""",24,2000-10-18,Jackson State
2535,4568320,28,Timothy,McKay,practiceSquad,Guard,312,"6' 4""",23,2002-01-15,NC State
2536,4362858,28,Darius,Rush,practiceSquad,Cornerback,198,"6' 2""",25,2000-02-22,South Carolina


Question #1

In [13]:
conn = sqlite3.connect("nfl.db")

# Top fantasy performers
df = pd.read_sql("""
SELECT
    position,
    ROUND(AVG(CAST(SUBSTR(weight, 1, INSTR(weight, ' ') - 1) AS REAL)), 2) AS avg_weight_lbs
FROM players
WHERE position IN ('QB', 'WR', 'DE')
GROUP BY position;
""", conn)
df

,position,avg_weight_lbs
0,DE,0.0
1,QB,0.0
2,WR,0.0
